In [1]:
import time
import numpy as np
from snnpy import *
from sklearn.neighbors import BallTree
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
from bf_search import *

In [2]:
def bvecs_read(fname):
    a = np.fromfile(fname, dtype=np.int32, count=1)
    b = np.fromfile(fname, dtype=np.uint8)
    d = a[0]
    return b.reshape(-1, d + 4)[:, 4:].copy()


def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read2(fname):
    return ivecs_read(fname).view('float32')

### siftsmall

In [3]:
sift_train = fvecs_read2('Euclidean_data/siftsmall/siftsmall_learn.fvecs')
sift_query = fvecs_read2('Euclidean_data/siftsmall/siftsmall_query.fvecs')


sift_kdtree_index_timing = list()
sift_balltree_index_timing = list()
sift_snn_index_timing = list()

sift_bf_run_timing1 = list()
sift_bf_run_timing2 = list()
sift_kdtree_run_timing = list()
sift_balltree_run_timing = list()
sift_snn_run_timing = list()


In [4]:
radius =  [210, 230, 250, 270, 290]  # test various radius

In [5]:
# Brute force 1
for R in radius:
    st = time.time()
    neigh = NearestNeighbors(radius=R, algorithm='brute')
    neigh.fit(sift_train)
    for j in range(sift_query.shape[0]): 
        ind = neigh.radius_neighbors(
           sift_query[j:j+1], radius=R, return_distance=False
        )
    et = time.time() - st
    sift_bf_run_timing1.append(et)
    print("brute force 1 query time:", et)
print()
    

# Brute force 2
for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]): 
        ind = bf_radius_fairness(sift_query[j], sift_train, R, return_distance=False)
    et = time.time() - st
    sift_bf_run_timing2.append(et)
    print("brute force 2 query time:", et)
print()
    
    
# KDtree
st = time.time()
tree = KDTree(sift_train)   
et = time.time() - st
sift_kdtree_index_timing.append(et)
print("kdtree build time:", et)
for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = tree.query_radius(sift_query[j:j+1], r=R, return_distance=False)
    et = time.time() - st
    sift_kdtree_run_timing.append(et)
    print("kdtree query time:", et)
print()


# Balltree
st = time.time()
tree = BallTree(sift_train)   
et = time.time() - st
sift_balltree_index_timing.append(et)
print("balltree build time:", et)

for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = tree.query_radius(sift_query[j:j+1], r=R, return_distance=False)
    et = time.time() - st
    sift_balltree_run_timing.append(et)
    print("balltree query time:", et)

print()


# SNN
st = time.time()
snn = build_snn_model(sift_train)
et = time.time() - st
sift_snn_index_timing.append(et)
print("snn build time:", et)


for R in radius:
    st = time.time()
    for j in range(sift_query.shape[0]):
        ind = snn.query_radius(sift_query[j], R)
    et = time.time() - st
    sift_snn_run_timing.append(et)
    print("snn query time:", et)

brute force 1 query time: 1.9035966396331787
brute force 1 query time: 2.1556832790374756
brute force 1 query time: 2.124293327331543
brute force 1 query time: 2.297062873840332
brute force 1 query time: 2.2063982486724854

brute force 2 query time: 0.41866207122802734
brute force 2 query time: 0.4152505397796631
brute force 2 query time: 0.4545729160308838
brute force 2 query time: 0.4278860092163086
brute force 2 query time: 0.42761874198913574

kdtree build time: 0.7205452919006348
kdtree query time: 1.5881330966949463
kdtree query time: 1.8584811687469482
kdtree query time: 1.8352632522583008
kdtree query time: 1.9908971786499023
kdtree query time: 1.9859070777893066

balltree build time: 0.6620845794677734
balltree query time: 1.2548103332519531
balltree query time: 1.374887466430664
balltree query time: 1.3145861625671387
balltree query time: 1.4727535247802734
balltree query time: 1.5332233905792236

snn build time: 0.07909703254699707
snn query time: 0.11117815971374512
snn que

In [6]:
with open('result/siftsmall_kdtree_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_kdtree_index_timing))

with open('result/siftsmall_balltree_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_balltree_index_timing))

with open('result/siftsmall_snn_index_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_snn_index_timing))
    
with open('result/siftsmall_bf_run_timing1_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_bf_run_timing1))

with open('result/siftsmall_bf_run_timing2_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_bf_run_timing2))
    
with open('result/siftsmall_kdtree_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_kdtree_run_timing))

with open('result/siftsmall_balltree_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_balltree_run_timing))

with open('result/siftsmall_snn_run_timing_norm.npy', 'wb') as f:
    np.save(f, np.array(sift_snn_run_timing))
    